In [2]:
!pip install peft

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/411.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/411.1 kB ? eta -:--:--
    --------------------------------------- 10.2/411.1 kB ? eta -:--:--
   -- ------------------------------------ 30.7/411.1 kB 262.6 kB/s eta 0:00:02
   --- ----------------------------------- 41.0/411.1 kB 279.3 kB/s eta 0:00:02
   ---------------------------------------  409.6/411.1 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 411.1/411.1 kB 2.1 MB/s eta 0:00:00


In [4]:
!pip install trl

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/348.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/348.0 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/348.0 kB 653.6 kB/s eta 0:00:01
   -------------------------------------- - 337.9/348.0 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 348.0/348.0 kB 3.1 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
import json

In [6]:
# 경로 설정
BASE_MODEL = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
DATA_PATH = "./classified_Empathy_Supervisor.json"  # 업로드된 JSON 파일 경로
OUTPUT_DIR = "./lora_adapters/empathy"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--MLP-KTLim--llama-3-Korean-Bllossom-8B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # 필요시 "k_proj", "o_proj"도 포함
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)

In [ ]:
with open(DATA_PATH, "r", encoding="utf-8") as f:
    json_data = json.load(f)

train_data = [
    {
        "instruction": "아래 내용을 감정적으로 공감하며 응답하세요.",
        "input": "",
        "output": item["content"]
    }
    for item in json_data if item.get("Agent") == "Empathy Supervisor"
]

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

In [ ]:
def formatting_func(example):
    return f"Instruct: {example['instruction']}\nInput: {example['input']}\nOutput: {example['output']}"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_data,
    formatting_func=formatting_func,
    dataset_text_field="output"
)

trainer.train()

In [ ]:
model.save_pretrained(OUTPUT_DIR)
